In [1]:
path_to_file = "G:/num.txt"

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 478552 characters


In [4]:
print(text[:250])

نوید 
بهاره 
شیوا
پری 
نیلوفر
بهنام 
حمیده
موژان 
مهسا 
دانیال
مینا 
امیر 
حامد
سهی 
سید مهدی
شقایق 
سید آرمین 
نوشین 
پریسا
فرامرز
شیما سادات 
علیرضا
غزل 
پروا
میلاد 
مهسا
مهدی
عسل
یاسمن 
الناز
سیده سمانه
یاسمین
حورا


In [5]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

102 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  '-' :   6,
  '.' :   7,
  '/' :   8,
  '0' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ';' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'نوید \r\nبهاره ' ---- characters mapped to int ---- > [ 84  86 100  66   2   1   0  59  85  58  68  85   2]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

ن
و
ی
د
 


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
    print("***"*5)

'نوید \r\nبهاره \r\nشیوا\r\nپری \r\nنیلوفر\r\nبهنام \r\nحمیده\r\nموژان \r\nمهسا \r\nدانیال\r\nمینا \r\nامیر \r\nحامد\r\nسهی \r\nسی'
***************
'د مهدی\r\nشقایق \r\nسید آرمین \r\nنوشین \r\nپریسا\r\nفرامرز\r\nشیما سادات \r\nعلیرضا\r\nغزل \r\nپروا\r\nمیلاد \r\nمهسا\r\nمهد'
***************
'ی\r\nعسل\r\nیاسمن \r\nالناز\r\nسیده سمانه\r\nیاسمین\r\nحورا\r\nآتوسا\r\nامید \r\nمریم\r\nرویا \r\nسجاد\r\nابراهیم \r\nسیده یاسم'
***************
'ن \r\nسپیده \r\nامیر حسین \r\nفضیلت \r\nسیدکیارش \r\nدنیا\r\nفریناز \r\nمحمدرضا\r\nریحانه\r\nمهستی سادات \r\nآیدا\r\nمیلاد\r'
***************
'\nفاطمه \r\nمحمد امین \r\nساناز \r\nحانیه \r\nمریم \r\nآرمان \r\nعرفان\r\nبهنام\r\nهلیا \r\nآرین \r\nملیکا\r\nنسرین\r\nپروین \r'
***************


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'نوید \r\nبهاره \r\nشیوا\r\nپری \r\nنیلوفر\r\nبهنام \r\nحمیده\r\nموژان \r\nمهسا \r\nدانیال\r\nمینا \r\nامیر \r\nحامد\r\nسهی \r\nس'
Target data: 'وید \r\nبهاره \r\nشیوا\r\nپری \r\nنیلوفر\r\nبهنام \r\nحمیده\r\nموژان \r\nمهسا \r\nدانیال\r\nمینا \r\nامیر \r\nحامد\r\nسهی \r\nسی'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 84 ('ن')
  expected output: 86 ('و')
Step    1
  input: 86 ('و')
  expected output: 100 ('ی')
Step    2
  input: 100 ('ی')
  expected output: 66 ('د')
Step    3
  input: 66 ('د')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 1 ('\r')


In [14]:
# Batch size
BATCH_SIZE = 32


dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(32, 100), dtype=tf.int32, name=None), TensorSpec(shape=(32, 100), dtype=tf.int32, name=None))>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 25

# Number of RNN units
rnn_units = 1024

# LSTM  model 

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
     
    return model


In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model.predict(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

1/1 [==============================] - 1s 684ms/step
(32, 100, 102) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 25)            2550      
                                                                 
 lstm (LSTM)                 (32, None, 1024)          4300800   
                                                                 
 dense (Dense)               (32, None, 102)           104550    
                                                                 
Total params: 4,407,900
Trainable params: 4,407,900
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()




In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam',loss=loss)

In [22]:
sampled_indices

array([ 55,  25,  58,  40, 101,  36, 101, 101,  45,  94,  58,  88,  93,
        57,  98,  95,  49,  47,   8,  12,  88,  47,  62,  98,  65,  10,
        70,  38,  90,  11,  31,  96,  38,  75,  58,  33,  87,  89,  16,
        57,  88,  72,  36,  20,  22,  94,  28,  36,  71,  84,  78,  79,
        49,  34,  16,  66,  22,  18,   1,  78, 101,  84,   3,  68,  58,
        67,  11,  22,  16,   5,  16,  49,  59,  60,  69,  56,  57, 100,
        82,  26,  48,  15,  45,  67,  95,  91,  61,  17,  86,  83,  38,
        48,  27,  41,  38,  19,  59,  69,  41,  48], dtype=int64)

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'نوید \r\nبهاره \r\nشیوا\r\nپری \r\nنیلوفر\r\nبهنام \r\nحمیده\r\nموژان \r\nمهسا \r\nدانیال\r\nمینا \r\nامیر \r\nحامد\r\nسهی \r\nس'

Next Char Predictions: 
 "أLاn\u200ci\u200c\u200ctٔاًّئکپ\xa0v/3ًvثکخ1سlَ2dچlظاfيٍ7ئًصi?DٔUiشنـف\xa0g7دD9\rـ\u200cن'راذ2D7)7\xa0بةزؤئیلNy6tذپُت8ومlyOol;بزoy"


In [24]:

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=40

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
148/148 [==============================] - 443s 3s/step - loss: 3.0948
Epoch 2/40
148/148 [==============================] - 400s 3s/step - loss: 2.6207
Epoch 3/40
148/148 [==============================] - 396s 3s/step - loss: 2.4838
Epoch 4/40
148/148 [==============================] - 405s 3s/step - loss: 2.4283
Epoch 5/40
148/148 [==============================] - 420s 3s/step - loss: 2.3826
Epoch 6/40
148/148 [==============================] - 419s 3s/step - loss: 2.3373
Epoch 7/40
148/148 [==============================] - 418s 3s/step - loss: 2.2849
Epoch 8/40
148/148 [==============================] - 414s 3s/step - loss: 2.2286
Epoch 9/40
148/148 [==============================] - 406s 3s/step - loss: 2.1649
Epoch 10/40
148/148 [==============================] - 490s 3s/step - loss: 2.0979
Epoch 11/40
148/148 [==============================] - 447s 3s/step - loss: 2.0302
Epoch 12/40
148/148 [==============================] - 428s 3s/step - loss: 1.9636
Epoch 13/40
1

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_40'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []


  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions 
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u"امیر"))

امیری
پائيزی
میر مددحقيقي
قشوادی پورجهان 
فرهودی
عارفه نوری
شهامی سروستاني
زینالي فر
مطلبی پور
رجبي فچا
بنائی شوطهري
مدنی
عادلخانی
کرمی رباب ارام
ژرگر بامله
دن آرا
پوریافي 
روز بهنا 
عبادی
فاضل اردبيلي
کاشگرور مهری
ابندي مطاع 
وادگاني
مروت قاسمی 
مهدی زاده 
مشاوریان تهوهه
زنانی
امیدی گرماني 
حسینعلی وانجان
اوجیانی
بازین دلد
محمدی خورده
یاراحمدی
كیانی فرد
خامران زماني
فجود لوله لو
عبادفرد
وادظافري
اخوان مهر
ملیحي
عنیدی مقدم
بهشتي زرنقی
رهیدی دريزي
ميرشرامیان
جاهدمنش
فردخي 
قدمدی تبریز
گل گین اوغلی
ناصری پور
امیدی طرقی
میرفخرایی شريفی
میرباقري فير آبادی
باحجب فردوس
عرب زاده ارونقي زاده
کاوه محمدی
اسماعیل خان قصري
گودرزيان
رمضان زاده طوسه
محجر قنبري
مامانی 
اردلی جاد
امامعلی زاده
اطهري
پژوعلی
پورحیدری
اندوهی
موسوی مشکوی
خمیعیان
جنوبدی
عمادورد
نوسخيابانیان
رجبيعراق محمدی
جوانمرد اول
عبودی حسامی
ماهوتچیان
خمیاي
عزيززاده شريفي
ميرزاي پیشگاه آبادو
امام لاعی
 شهسواری گودرزابی
حلاج
جمالباز 
ولدي
علائيان
امیرحسنی
زم
